In [ ]:
import re
from settings import *

text = "\n   aa   e483zigko zez\n\nef\n j •j •  j.  dz...  '$;.  \n<eom><eod>\n\n<eom>  \t  %kzeé i3zçr zg   35  ¨e:=zge \ng   ezù5 =   = zae 5gez $gez  \n"

text = text.replace('\n', '<nl>').replace('\t', '    ')

# Start

# Split control characters

words = [text]
temp = []

for control in CONTROL_CHARS:
	for i in range(len(words)):

		parts = words[i].split(control)

		for part in parts:
			if len(part) > 0:
				temp.append(part)

			temp.append(control)

		if len(parts) > 0:
			temp.pop()

	words = temp
	temp = []

# Split words

for i in range(len(words)):

	if words[i] in CONTROL_CHARS:
		temp.append(words[i])
		continue

	word = ''

	for j in range(len(words[i])):

		if len(word) == 0:
			word += words[i][j]
			continue

		if word[-1] != ' ' and words[i][j] == ' ':
			temp.append(word)
			word = words[i][j]
			continue

		if word[-1].isalpha() and not words[i][j].isalpha():
			temp.append(word)
			word = words[i][j]
			continue

		if word[-1] != ' ' and not word[-1].isalpha() and words[i][j].isalpha():
			temp.append(word)
			word = words[i][j]
			continue

		word += words[i][j]

	if len(word) > 0:
		temp.append(word)

words = temp
temp = []

# Split multiple spaces

for i in range(len(words)):

	j = 0

	while j < len(words[i]) and words[i][j] == ' ':
		j += 1

	if j > 1 and j < len(words[i]):
		temp.append(words[i][:j - 1])
		temp.append(words[i][j - 1:])

	else:
		temp.append(words[i])

words = temp
temp = []

# Stop

print(text.replace(' ', '_'))

for i in range(len(words)):
	if len(words[i]) == 0:
		print('\033[92m_\033[0m', end = '')
	if i % 2 == 1:
		print('\033[91m' + words[i].replace(' ', '_') + '\033[0m', end = '')
	else:
		print('\033[94m' + words[i].replace(' ', '_') + '\033[0m', end = '')

In [ ]:
print('a.n'.split('.'))

In [ ]:
import time
import random
import string

def get_random_string():

	return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(10))

a = {}

for i in range(1000000):
	a[get_random_string()] = 0

a['aeqhrtherqth'] = 0
b = {'aeqhrtherqth': 0}

In [ ]:
start = time.time()
print('aeqhrtherqth' in a)
end = time.time()
print(end - start)

start = time.time()
print('aeqhrtherqth' in b)
end = time.time()
print(end - start)

In [2]:
import re
import regex
import utils
from settings import *
a = "   aa   e483zigko ze<nl>zef j <nl>•j •<nl>  j.  dz...ze____%ç __ '$;.  <eom><eod><eom> a___ef__  >=0 %kzeé i3zçr zg   35  ¨e:=zge g   ezù5 =   = zae 5gez $gez   "

# b = regex.split('(' + '|'.join(CONTROL_CHARS) + ')', a, flags = regex.UNICODE)
# b = list(filter(None, b))

# temp = []

# for i in b:

# 	if i in CONTROL_CHARS:
# 		temp.append(i)
# 		continue

# 	t = regex.split(r'(\d+|\s+|\p{L}+|[^\d\p{L}\s]+)', i, flags = regex.UNICODE)
# 	t = list(filter(None, t))
# 	temp.extend(t)

# b = temp
reg = r'(' + r'|'.join(CONTROL_CHARS) + r'|\d+|\s+|\p{L}+|[^\d\p{L}\s' + r''.join([f'[{i}]' for i in CONTROL_CHARS]) + r']+)'
b = regex.split(reg, a, flags = regex.UNICODE)
b = list(filter(None, b))

'''
temp = []
i = 0

while i < len(b) - 1:

	if b[i] == ' ':
		temp.append(' ' + b[i + 1])
		i += 2
		continue

	if b[i].endswith(' '):
		temp.extend([b[i][:-1], ' ' + b[i + 1]])
		i += 2
		continue

	temp.append(b[i])
	i += 1

temp.append(b[-1])

b = temp
'''
print(a)
utils.print_tokens(b)

   aa   e483zigko ze<nl>zef j <nl>•j •<nl>  j.  dz...ze____%ç __ '$;.  <eom><eod><eom> a___ef__  >=0 %kzeé i3zçr zg   35  ¨e:=zge g   ezù5 =   = zae 5gez $gez   
___aa___e483zigko_ze<nl>zef_j_<nl>•j_•<nl>__j.__dz...ze____%ç____'$;.__<eom><eod><eom>_a___ef____>=0_%kzeé_i3zçr_zg___35__¨e:=zge_g___ezù5_=___=_zae_5gez_$gez___


In [ ]:
print("0".isdecimal())